In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np

# Testing Alabama

In [2]:
response = requests.get('https://www.fjc.gov/history/courts/u.s.-district-courts-districts-alabama-judges-all-districts')
doc = BeautifulSoup(response.text)

In [3]:
tables = doc.find_all(class_='field-content')
for table in tables:
    row = {}
    row['Judge'] = table.find('a').text
    row['URL'] = 'https://www.fjc.gov' + table.find('a')['href']
    print(row)
    print('---------')

{'Judge': 'Tait, Charles', 'URL': 'https://www.fjc.gov/history/judges/tait-charles'}
---------
{'Judge': 'Crawford, William', 'URL': 'https://www.fjc.gov/history/judges/crawford-william'}
---------
{'Judge': 'Gayle, John', 'URL': 'https://www.fjc.gov/history/judges/gayle-john'}
---------
{'Judge': 'Jones, William Giles', 'URL': 'https://www.fjc.gov/history/judges/jones-william-giles'}
---------
{'Judge': 'Lane, George Washington', 'URL': 'https://www.fjc.gov/history/judges/lane-george-washington'}
---------
{'Judge': 'Busteed, Richard', 'URL': 'https://www.fjc.gov/history/judges/busteed-richard'}
---------
{'Judge': 'Bruce, John', 'URL': 'https://www.fjc.gov/history/judges/bruce-john'}
---------
{'Judge': 'Toulmin, Harry Theophilus', 'URL': 'https://www.fjc.gov/history/judges/toulmin-harry-theophilus'}
---------
{'Judge': 'Jones, Thomas Goode', 'URL': 'https://www.fjc.gov/history/judges/jones-thomas-goode'}
---------
{'Judge': 'Hundley, Oscar Richard', 'URL': 'https://www.fjc.gov/histo

AttributeError: 'NoneType' object has no attribute 'text'

In [4]:
rows = []
tables = doc.find_all(class_='field-content')
for table in tables:
    row = {}
    row['Judge'] = table.find('a').text
    row['URL'] = 'https://www.fjc.gov' + table.find('a')['href']
    rows.append(row)

AttributeError: 'NoneType' object has no attribute 'text'

In [5]:
df = pd.DataFrame(rows)
df.head()

,Judge,URL
0,"Tait, Charles",https://www.fjc.gov/history/judges/tait-charles
1,"Crawford, William",https://www.fjc.gov/history/judges/crawford-wi...
2,"Gayle, John",https://www.fjc.gov/history/judges/gayle-john
3,"Jones, William Giles",https://www.fjc.gov/history/judges/jones-willi...
4,"Lane, George Washington",https://www.fjc.gov/history/judges/lane-george...


In [6]:
df['Slug'] = df.URL.str.replace("https://www.fjc.gov/", "")
df.head()

,Judge,URL,Slug
0,"Tait, Charles",https://www.fjc.gov/history/judges/tait-charles,history/judges/tait-charles
1,"Crawford, William",https://www.fjc.gov/history/judges/crawford-wi...,history/judges/crawford-william
2,"Gayle, John",https://www.fjc.gov/history/judges/gayle-john,history/judges/gayle-john
3,"Jones, William Giles",https://www.fjc.gov/history/judges/jones-willi...,history/judges/jones-william-giles
4,"Lane, George Washington",https://www.fjc.gov/history/judges/lane-george...,history/judges/lane-george-washington


In [7]:
response = requests.get('https://www.fjc.gov/history/judges/gayle-john')
doc = BeautifulSoup(response.text)

In [8]:
judge_info = doc.find(class_='field__item even')

In [9]:
judge_info.text

'Born September 11, 1792,  in Sumter County, SCDied July 21, 1859,  in Jacksonville, ALFederal Judicial Service: Judge, U.S. District Court for the Middle District of AlabamaNominated by Zachary Taylor on March 12, 1849, to a seat vacated by William Crawford. Confirmed by the Senate on March 13, 1849, and received commission on March 13, 1849. Service terminated on July 21, 1859, due to death.Judge, U.S. District Court for the Northern District of AlabamaNominated by Zachary Taylor on March 12, 1849, to a seat vacated by William Crawford. Confirmed by the Senate on March 13, 1849, and received commission on March 13, 1849. Service terminated on July 21, 1859, due to death.Judge, U.S. District Court for the Southern District of AlabamaNominated by Zachary Taylor on March 12, 1849, to a seat vacated by William Crawford. Confirmed by the Senate on March 13, 1849, and received commission on March 13, 1849. Service terminated on July 21, 1859, due to death.Education: South Carolina College 

# Regular expressions for each judge's bio

In [10]:
Born = re.search(r'Born\s(?:[A-Z][a-z]*\s\d*,\s)?(\d\d\d\d)', judge_info.text)

Died = re.search(r'Died\s[A-Z][a-z]*\s\d*,\s(\d\d\d\d),\s', judge_info.text)

Court = re.search(r'Federal Judicial Service: Judge,\s(.*?)Nominated', judge_info.text)

President = re.search(r'Nominated by ([A-Z][a-z]*\s[A-Z]?[.]?[A-Z]?[.]?\s?[A-Z][a-z]*)', judge_info.text)

Confirmed = re.search(r'Confirmed by the Senate on ([A-Z][a-z]*\s\d*,\s\d\d\d\d)', judge_info.text)

Education = re.search(r'Education:\s(.*)Professional', judge_info.text)

Termination = re.search(r'due to (.*).Education', judge_info.text)

In [11]:
def scrape_page(row):
    
    url = f"https://www.fjc.gov/{row['Slug']}"    
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    
    judge_info = doc.find(class_='field__item even')

    page = {}
    
    page['Born'] = re.findall(r'Born\s(?:[A-Z][a-z]*\s\d*,\s)?(\d\d\d\d)', judge_info.text)
    page['Died'] = re.findall(r'Died\s[A-Z][a-z]*\s\d*,\s(\d\d\d\d),\s', judge_info.text)
    page['Court'] = re.findall(r'Federal Judicial Service: Judge,\s(.*?)Nominated', judge_info.text)
    page['President'] = re.findall(r'Nominated by ([A-Z][a-z]*\s[A-Z]?[.]?[A-Z]?[.]?\s?[A-Z][a-z]*)', judge_info.text)
    page['Confirmed'] = re.findall(r'Confirmed by the Senate on ([A-Z][a-z]*\s\d*,\s\d\d\d\d)', judge_info.text)
    page['Education'] = re.findall(r'Education:\s(.*)Professional', judge_info.text)
    page['Termination'] = re.findall(r'due to (.*).Education', judge_info.text)


    return pd.Series(page)

In [12]:
scrape_page(df.loc[0])

Born                                                      [1768]
Died                                                      [1835]
Court          [U.S. District Court for the District of Alabama]
President                                         [James Monroe]
Confirmed                                         [May 13, 1820]
Education                                       [Read law, 1795]
Termination    [reassignment.Judge, U.S. District Court for t...
dtype: object

In [13]:
scraped_df = df.apply(scrape_page, axis=1)

In [14]:
df = df.merge(scraped_df, left_index=True, right_index = True)
df.head()

,Judge,URL,Slug,Born,Died,Court,President,Confirmed,Education,Termination
0,"Tait, Charles",https://www.fjc.gov/history/judges/tait-charles,history/judges/tait-charles,[1768],[1835],[U.S. District Court for the District of Alabama],[James Monroe],"[May 13, 1820]","[Read law, 1795]","[reassignment.Judge, U.S. District Court for t..."
1,"Crawford, William",https://www.fjc.gov/history/judges/crawford-wi...,history/judges/crawford-william,[1784],[1849],[U.S. District Court for the Northern District...,"[John Quincy, John Quincy]","[May 22, 1826, May 22, 1826]",[Read lawHampden-Sydney College],"[death.Judge, U.S. District Court for the Sout..."
2,"Gayle, John",https://www.fjc.gov/history/judges/gayle-john,history/judges/gayle-john,[1792],[1859],[U.S. District Court for the Middle District o...,"[Zachary Taylor, Zachary Taylor, Zachary Taylor]","[March 13, 1849, March 13, 1849, March 13, 1849]",[South Carolina College (now University of Sou...,"[death.Judge, U.S. District Court for the Nort..."
3,"Jones, William Giles",https://www.fjc.gov/history/judges/jones-willi...,history/judges/jones-william-giles,[1808],[1883],[],[],"[January 30, 1860, January 30, 1860, January 3...",[Hampden-Sydney CollegeUniversity of VirginiaR...,"[resignation.Judge, U.S. District Court for th..."
4,"Lane, George Washington",https://www.fjc.gov/history/judges/lane-george...,history/judges/lane-george-washington,[1806],[1863],[U.S. District Court for the Middle District o...,"[Abraham Lincoln, Abraham Lincoln, Abraham Lin...","[March 28, 1861, March 28, 1861, March 28, 1861]",[Read law],"[death.Judge, U.S. District Court for the Nort..."


# Loop through all states

In [15]:
df_states = pd.read_excel("list_of_states copy.xlsx")

In [16]:
df_states['Slug'] = df_states.URL.str.replace("https://www.fjc.gov/", "")
df_states.head()

,State,URL,Slug
0,Alabama,https://www.fjc.gov/history/courts/u.s.-distri...,history/courts/u.s.-district-courts-districts-...
1,Alaska,https://www.fjc.gov/history/courts/u.s.-distri...,history/courts/u.s.-district-court-district-al...
2,Arizona,https://www.fjc.gov/history/courts/u.s.-distri...,history/courts/u.s.-district-court-district-ar...
3,Arkansas,https://www.fjc.gov/history/courts/u.s.-distri...,history/courts/u.s.-district-courts-districts-...
4,California,https://www.fjc.gov/history/courts/u.s.-distri...,history/courts/u.s.-district-courts-districts-...


In [17]:
urls = list(df_states.URL)

In [18]:
rows_judges = []

for url in urls:

    response = requests.get(url)
    doc1 = BeautifulSoup(response.text)


    judges_links = doc1.find_all(class_='field-content')
    for judge in judges_links:

        row = {}
        try:
            row['Judge'] = judge.find('a').text
        except:
            pass
        try:
            row['URL'] = 'https://www.fjc.gov' + judge.find('a')['href']
        except:
            pass

        rows_judges.append(row)

In [19]:
df_all_judges = pd.DataFrame(rows_judges)

In [20]:
df_all_judges['Slug'] = df_all_judges.URL.str.replace("https://www.fjc.gov/", "")

In [21]:
def all_judges(row):
    
    url = f"https://www.fjc.gov/{row['Slug']}"    
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    
    judge_info = doc.find(class_='field__item even')

    page = {}
    try:
        page['Born'] = re.findall(r'Born\s(?:[A-Z][a-z]*\s\d*,\s)?(\d\d\d\d)', judge_info.text)
    except:
        pass
    try:
        page['Died'] = re.findall(r'Died\s[A-Z][a-z]*\s\d*,\s(\d\d\d\d),\s', judge_info.text)
    except:
        pass
    try:
        page['Court'] = re.findall(r'Federal Judicial Service: Judge,\s(.*?)Nominated', judge_info.text)
    except:
        pass
    try:
        page['President'] = re.findall(r'Nominated by ([A-Z][a-z]*\s[A-Z]?[.]?[A-Z]?[.]?\s?[A-Z][a-z]*)', judge_info.text)
    except:
        pass
    try:
        page['Confirmed'] = re.findall(r'Confirmed by the Senate on ([A-Z][a-z]*\s\d*,\s\d\d\d\d)', judge_info.text)
    except:
        pass
    try:
        page['Education'] = re.findall(r'Education:\s(.*)Professional', judge_info.text)
    except:
        pass
    try:
        page['Termination'] = re.findall(r'due to (.*).Education', judge_info.text)
    except:
        pass

    return pd.Series(page)

In [22]:
scraped_judges = df_all_judges.apply(all_judges, axis=1)

In [23]:
df_final = df_all_judges.merge(scraped_judges, left_index=True, right_index = True)

In [25]:
df_final.head()

,Judge,URL,Slug,Born,Confirmed,Court,Died,Education,President,Termination
0,"Tait, Charles",https://www.fjc.gov/history/judges/tait-charles,history/judges/tait-charles,[1768],"[May 13, 1820]",[U.S. District Court for the District of Alabama],[1835],"[Read law, 1795]",[James Monroe],"[reassignment.Judge, U.S. District Court for t..."
1,"Crawford, William",https://www.fjc.gov/history/judges/crawford-wi...,history/judges/crawford-william,[1784],"[May 22, 1826, May 22, 1826]",[U.S. District Court for the Northern District...,[1849],[Read lawHampden-Sydney College],"[John Quincy, John Quincy]","[death.Judge, U.S. District Court for the Sout..."
2,"Gayle, John",https://www.fjc.gov/history/judges/gayle-john,history/judges/gayle-john,[1792],"[March 13, 1849, March 13, 1849, March 13, 1849]",[U.S. District Court for the Middle District o...,[1859],[South Carolina College (now University of Sou...,"[Zachary Taylor, Zachary Taylor, Zachary Taylor]","[death.Judge, U.S. District Court for the Nort..."
3,"Jones, William Giles",https://www.fjc.gov/history/judges/jones-willi...,history/judges/jones-william-giles,[1808],"[January 30, 1860, January 30, 1860, January 3...",[],[1883],[Hampden-Sydney CollegeUniversity of VirginiaR...,[],"[resignation.Judge, U.S. District Court for th..."
4,"Lane, George Washington",https://www.fjc.gov/history/judges/lane-george...,history/judges/lane-george-washington,[1806],"[March 28, 1861, March 28, 1861, March 28, 1861]",[U.S. District Court for the Middle District o...,[1863],[Read law],"[Abraham Lincoln, Abraham Lincoln, Abraham Lin...","[death.Judge, U.S. District Court for the Nort..."


In [ ]:
df_final.to_csv('Final_Project2.csv', index=False)